In [3]:
from langchain_community.graphs import Neo4jGraph
import os
from dotenv import load_dotenv
load_dotenv()
# Warning control
import warnings
warnings.filterwarnings("ignore")

In [4]:
azure_openai_api_key = os.environ["OPENAI_API_KEY"]
azure_openai_endpoint = os.environ["OPENAI_API_BASE"]

In [5]:
from openai import AzureOpenAI

client = AzureOpenAI(
  api_key = azure_openai_api_key,  
  api_version = "2023-07-01-preview",
  azure_endpoint = azure_openai_endpoint
)
def embed_text(text):
    response = client.embeddings.create(
    input = text,
    model= "text-embedding-ada-002"
    )
    return response.data[0].embedding

In [6]:
def embed_text(text):
    response = client.embeddings.create(
    input = text,
    model= "text-embedding-ada-002"
    )
    return response.data[0].embedding

In [7]:
NEO4J_URI = "bolt://localhost:7687"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "12345678"
NEO4J_DATABASE = 'neo4j'

In [8]:
graph = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE)

In [9]:
question = "What movies are about love?"

In [10]:
question_embedding = embed_text(question)
question_embedding[:10]

[-0.004169964697211981,
 -0.030407307669520378,
 0.0007921198266558349,
 -0.02763153612613678,
 -0.007961414754390717,
 0.0019603881519287825,
 -0.00657983822748065,
 -0.02148698829114437,
 -0.00205501657910645,
 -0.007980341091752052]

**Perform similarity search**

In [11]:
graph.query("""
    with $question_embedding as question_embedding
    CALL db.index.vector.queryNodes(
        'movie_tagline_embeddings', 
        $top_k, 
        question_embedding
        ) YIELD node AS movie, score
    RETURN movie.title, movie.tagline, score
    """,
    params={
        "question_embedding": question_embedding,
        "top_k": 5
    })

[{'movie.title': 'Heat',
  'movie.tagline': 'A Los Angeles crime saga',
  'score': 0.8907766342163086},
 {'movie.title': 'Grumpier Old Men',
  'movie.tagline': 'Still Yelling. Still Fighting. Still Ready for Love.',
  'score': 0.887347400188446},
 {'movie.title': 'Tom and Huck',
  'movie.tagline': 'The Original Bad Boys.',
  'score': 0.8832508325576782},
 {'movie.title': 'Sense and Sensibility',
  'movie.tagline': 'Lose your heart and come to your senses.',
  'score': 0.8757961988449097},
 {'movie.title': 'Balto',
  'movie.tagline': 'Part Dog. Part Wolf. All Hero.',
  'score': 0.8753572702407837}]

In [12]:
question = "What movies are about adventure?"
question_embedding = embed_text(question)
question_embedding[:10]

[0.010714428499341011,
 -0.036056939512491226,
 -0.01811829023063183,
 -0.019606761634349823,
 -0.0062810927629470825,
 0.023982355371117592,
 -0.019516941159963608,
 -0.03374724090099335,
 0.008744769729673862,
 -0.00961090624332428]

In [13]:
graph.query("""
    with $question_embedding as question_embedding
    CALL db.index.vector.queryNodes(
        'movie_tagline_embeddings', 
        $top_k, 
        question_embedding
        ) YIELD node AS movie, score
    RETURN movie.title, movie.tagline, score
    """,
    params={
        "question_embedding": question_embedding,
        "top_k": 5
    })

[{'movie.title': 'Toy Story',
  'movie.tagline': 'The adventure takes off!',
  'score': 0.9301033020019531},
 {'movie.title': 'Cutthroat Island',
  'movie.tagline': 'The Course Has Been Set. There Is No Turning Back. Prepare Your Weapons. Summon Your Courage. Discover the Adventure of a Lifetime!',
  'score': 0.8929015398025513},
 {'movie.title': 'Tom and Huck',
  'movie.tagline': 'The Original Bad Boys.',
  'score': 0.8922421932220459},
 {'movie.title': 'Jumanji',
  'movie.tagline': 'Roll the dice and unleash the excitement!',
  'score': 0.8902815580368042},
 {'movie.title': 'Heat',
  'movie.tagline': 'A Los Angeles crime saga',
  'score': 0.8866440653800964}]